# A partir del ejemplo
 
https://shreyz-max.medium.com/finetuning-gpt3-ba07a10fa9d3

Adaptamos a nuestra base de datos ED
Ver Notebook en JokeSample

In [13]:
#download openai
#!pip install openai

In [1]:
# Cargar librerias
import openai
import pandas as pd
import string

In [3]:
# API KEY de openai
openai.api_key = 'here_the_api_key'

In [4]:
# Cargar la primera parte del set de entrenamiento
df1 = pd.read_csv('ED_train_p1.csv',sep=";")
#df1.head()
df1.tail()


,utterance,utterance2,emotion
32275,It was a marketing job,What traits do you have that make you think yo...,0
32276,On the first date date I expect nothing more t...,Its nice to be prepared,2
32277,Its nice to be prepared,Absolutely. Never fails,2
32278,Absolutely. Never fails,I will have to start doing that,2
32279,Absolutely. Never fails,I will have to start doing that,2


In [6]:
# Cargar la segunda parte del set de entrenamiento
df2 = pd.read_csv('ED_train_p2.csv',sep=";")
#df1.head()
df2.tail()
print(len(df2))

,utterance,utterance2,emotion
32355,Oh hey that's awesome! That is awesome right?,It is soooo awesome. We have been wanting a b...,3
32356,It is soooo awesome. We have been wanting a b...,That is awesome!!!! Congratulations!,3
32357,I woke up this morning to my wife telling me s...,Oh hey that's awesome! That is awesome right?,3
32358,Oh hey that's awesome! That is awesome right?,It is soooo awesome. We have been wanting a b...,3
32359,It is soooo awesome. We have been wanting a b...,That is awesome!!!! Congratulations!,3


In [13]:
print(len(df1))
print(len(df2))
print(len(df_ED))

32280
32360
64640


In [8]:
# Unir las BD's tomando solamente los turnos (utternace y utterance2)
df_ED = pd.concat([df1,df2])


                                               utterance  \
0      I remember going to see the fireworks with my ...   
1      Was this a friend you were in love with, or ju...   
2                    This was a best friend. I miss her.   
3                                    Where has she gone?   
4                                     We no longer talk.   
...                                                  ...   
32355     Oh hey that's awesome!  That is awesome right?   
32356  It is soooo awesome.  We have been wanting a b...   
32357  I woke up this morning to my wife telling me s...   
32358     Oh hey that's awesome!  That is awesome right?   
32359  It is soooo awesome.  We have been wanting a b...   

                                              utterance2  emotion  
0      Was this a friend you were in love with, or ju...        2  
1                    This was a best friend. I miss her.        2  
2                                    Where has she gone?        2  
3      

In [15]:
# Eliminamos la columna emotion
df_ED = df_ED.drop(['emotion'], axis=1)
df_ED.head()

,utterance,utterance2
0,I remember going to see the fireworks with my ...,"Was this a friend you were in love with, or ju..."
1,"Was this a friend you were in love with, or ju...",This was a best friend. I miss her.
2,This was a best friend. I miss her.,Where has she gone?
3,Where has she gone?,We no longer talk.
4,We no longer talk.,Oh was this something that happened because of...


In [20]:
# Cambiamos el nombre de las columna para adaptar el dataset a lo que necesita GPT3
df_ED.rename(columns={'utterance':'prompt',
                        'utterance2':'completion'},
               inplace=True)

df_ED.head()

,prompt,completion
0,I remember going to see the fireworks with my ...,"Was this a friend you were in love with, or ju..."
1,"Was this a friend you were in love with, or ju...",This was a best friend. I miss her.
2,This was a best friend. I miss her.,Where has she gone?
3,Where has she gone?,We no longer talk.
4,We no longer talk.,Oh was this something that happened because of...


In [21]:
#saving the new csv file
df_ED.to_csv('ED.csv')

In [22]:
#converting csv to jsonl format because that is the format GPT3 can be finetuned in
!yes | openai tools fine_tunes.prepare_data -f 'ED.csv' 

Analyzing...

- Based on your file extension, your file is formatted as a CSV file
- Your file contains 64640 prompt-completion pairs
- The input file should contain exactly two columns/keys per row. Additional columns/keys present are: ['Unnamed: 0']
- `completion` column/key should not contain empty strings. These are rows: [30548, 30549, 30550]
- There are 23 duplicated prompt-completion sets. These are rows: [16057, 16058, 32115, 32276, 32382, 32463, 39726, 43536, 43537, 44501, 44704, 44913, 45911, 46875, 48073, 48439, 50479, 50592, 50593, 51052, 64634, 64635, 64636]
- Your data does not contain a common separator at the end of your prompts. Having a separator string appended to the end of the prompt makes it clearer to the fine-tuned model where the completion should begin. See https://beta.openai.com/docs/guides/fine-tuning/preparing-your-dataset for more detail and examples. If you intend to do open-ended generation, then you should leave the prompts empty
- Your data does not c

In [23]:
#creating a file id from the jsonl file
# Parece que se queda colgado
with open("ED_prepared.jsonl") as f:
    response = openai.File.create(file=f, purpose='fine-tune')
    print(response)

{
  "bytes": 10768083,
  "created_at": 1669844649,
  "filename": "file",
  "id": "file-NZ6ExR6dig3ua6VbPwj0l3Qh",
  "object": "file",
  "purpose": "fine-tune",
  "status": "uploaded",
  "status_details": null
}


In [24]:
#the file id is then used to create finetune model
response = openai.FineTune.create(training_file="file-NZ6ExR6dig3ua6VbPwj0l3Qh", model='ada')
print(response)

{
  "created_at": 1669844848,
  "events": [
    {
      "created_at": 1669844848,
      "level": "info",
      "message": "Created fine-tune: ft-0ErXUrLbYOu7EBJULWIxrpc1",
      "object": "fine-tune-event"
    }
  ],
  "fine_tuned_model": null,
  "hyperparams": {
    "batch_size": null,
    "learning_rate_multiplier": null,
    "n_epochs": 4,
    "prompt_loss_weight": 0.01
  },
  "id": "ft-0ErXUrLbYOu7EBJULWIxrpc1",
  "model": "ada",
  "object": "fine-tune",
  "organization_id": "org-xSwUUXjHMLby2b8IceH3t8iK",
  "result_files": [],
  "status": "pending",
  "training_files": [
    {
      "bytes": 10768083,
      "created_at": 1669844649,
      "filename": "file",
      "id": "file-NZ6ExR6dig3ua6VbPwj0l3Qh",
      "object": "file",
      "purpose": "fine-tune",
      "status": "processed",
      "status_details": null
    }
  ],
  "updated_at": 1669844848,
  "validation_files": []
}


In [25]:
#this shows how much training has been done
# PRIMERA CONSULTA
# NO REEJECUTAR
response = openai.FineTune.retrieve(id="ft-0ErXUrLbYOu7EBJULWIxrpc1")
print(response)

{
  "created_at": 1669844848,
  "events": [
    {
      "created_at": 1669844848,
      "level": "info",
      "message": "Created fine-tune: ft-0ErXUrLbYOu7EBJULWIxrpc1",
      "object": "fine-tune-event"
    },
    {
      "created_at": 1669844853,
      "level": "info",
      "message": "Fine-tune costs $3.50",
      "object": "fine-tune-event"
    },
    {
      "created_at": 1669844854,
      "level": "info",
      "message": "Fine-tune enqueued. Queue number: 0",
      "object": "fine-tune-event"
    },
    {
      "created_at": 1669844865,
      "level": "info",
      "message": "Fine-tune started",
      "object": "fine-tune-event"
    }
  ],
  "fine_tuned_model": null,
  "hyperparams": {
    "batch_size": 128,
    "learning_rate_multiplier": 0.2,
    "n_epochs": 4,
    "prompt_loss_weight": 0.01
  },
  "id": "ft-0ErXUrLbYOu7EBJULWIxrpc1",
  "model": "ada",
  "object": "fine-tune",
  "organization_id": "org-xSwUUXjHMLby2b8IceH3t8iK",
  "result_files": [],
  "status": "running",

In [4]:
#this shows how much training has been done
response = openai.FineTune.retrieve(id="ft-0ErXUrLbYOu7EBJULWIxrpc1")
print(response)

{
  "created_at": 1669844848,
  "events": [
    {
      "created_at": 1669844848,
      "level": "info",
      "message": "Created fine-tune: ft-0ErXUrLbYOu7EBJULWIxrpc1",
      "object": "fine-tune-event"
    },
    {
      "created_at": 1669844853,
      "level": "info",
      "message": "Fine-tune costs $3.50",
      "object": "fine-tune-event"
    },
    {
      "created_at": 1669844854,
      "level": "info",
      "message": "Fine-tune enqueued. Queue number: 0",
      "object": "fine-tune-event"
    },
    {
      "created_at": 1669844865,
      "level": "info",
      "message": "Fine-tune started",
      "object": "fine-tune-event"
    },
    {
      "created_at": 1669845762,
      "level": "info",
      "message": "Fine-tune is in the queue. Queue number: 2",
      "object": "fine-tune-event"
    },
    {
      "created_at": 1669846823,
      "level": "info",
      "message": "Fine-tune is in the queue. Queue number: 1",
      "object": "fine-tune-event"
    },
    {
      "cr

# Probando el modelo GPT3 entrenado


In [3]:
# Cargar librerias
import openai

In [4]:
# API KEY de openai
openai.api_key = 'Here-the-API-KEY'

In [14]:
#Checking the inference
ft_model = "ada:ft-mabel-2022-11-30-22-47-42"
res = openai.Completion.create(model=ft_model, prompt="I don't like Christmas", max_tokens=25, temperature=1)
res

<OpenAIObject text_completion id=cmpl-6N6nSmWyy7DJJ69zccq1Cu13y0Miv at 0x1111d6d10> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": " but I do like halls. -> Hall steps sound great!\n\nHow did you end up getting it?\n\nFun"
    }
  ],
  "created": 1670965690,
  "id": "cmpl-6N6nSmWyy7DJJ69zccq1Cu13y0Miv",
  "model": "ada:ft-mabel-2022-11-30-22-47-42",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 25,
    "prompt_tokens": 5,
    "total_tokens": 30
  }
}

In [5]:
#Checking the inference
ft_model = "ada:ft-mabel-2022-11-30-22-47-42"
res = openai.Completion.create(model=ft_model, prompt="I hate ... ", max_tokens=18, temperature=1)
res

<OpenAIObject text_completion id=cmpl-6Ic7mHgvfS1LaMbLw2v03B0ICIz5X at 0x11a2c5350> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": "              > why xxx\n"
    }
  ],
  "created": 1669894474,
  "id": "cmpl-6Ic7mHgvfS1LaMbLw2v03B0ICIz5X",
  "model": "ada:ft-mabel-2022-11-30-22-47-42",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 18,
    "prompt_tokens": 4,
    "total_tokens": 22
  }
}

In [6]:
#Checking the inference
ft_model = "ada:ft-mabel-2022-11-30-22-47-42"
res = openai.Completion.create(model=ft_model, prompt="Knock Knock... ", max_tokens=18, temperature=1)
res

<OpenAIObject text_completion id=cmpl-6Ic8URaDOS1YBPOfRHbANiLetLU0b at 0x11a2c5410> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": " -> I am sure you are gonna win it, I just hope you do!!\n\n"
    }
  ],
  "created": 1669894518,
  "id": "cmpl-6Ic8URaDOS1YBPOfRHbANiLetLU0b",
  "model": "ada:ft-mabel-2022-11-30-22-47-42",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 18,
    "prompt_tokens": 5,
    "total_tokens": 23
  }
}

In [7]:
#Checking the inference
ft_model = "ada:ft-mabel-2022-11-30-22-47-42"
res = openai.Completion.create(model=ft_model, prompt="I want to die ", max_tokens=18, temperature=1)
res

<OpenAIObject text_completion id=cmpl-6Ic99CVn65CUDpOxtO6hEaGSvip02 at 0x11a2c5dd0> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": " -> oh wow that did sound scary! why was it so scary?\n\n120 on"
    }
  ],
  "created": 1669894559,
  "id": "cmpl-6Ic99CVn65CUDpOxtO6hEaGSvip02",
  "model": "ada:ft-mabel-2022-11-30-22-47-42",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 18,
    "prompt_tokens": 5,
    "total_tokens": 23
  }
}

In [9]:
#Checking the inference
ft_model = "ada:ft-mabel-2022-11-30-22-47-42"
res = openai.Completion.create(model=ft_model, prompt="My father dies. ", max_tokens=21, temperature=1)
res

<OpenAIObject text_completion id=cmpl-6IcANrP9SygyraSNx1yWL8MXV0O1P at 0x11a2e1710> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": " I don't have the right to say anything right now, I'm sad -> I'm"
    }
  ],
  "created": 1669894635,
  "id": "cmpl-6IcANrP9SygyraSNx1yWL8MXV0O1P",
  "model": "ada:ft-mabel-2022-11-30-22-47-42",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 18,
    "prompt_tokens": 5,
    "total_tokens": 23
  }
}

In [10]:
#Checking the inference
ft_model = "ada:ft-mabel-2022-11-30-22-47-42"
res = openai.Completion.create(model=ft_model, prompt="My father dies. ", max_tokens=21, temperature=1)
res

<OpenAIObject text_completion id=cmpl-6IcCcjUV2SnZBQbiWKtU4Wor4plkC at 0x11a2e1bf0> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": " Then will you be responsible for him? -> I will be, but I'm going to be fine."
    }
  ],
  "created": 1669894774,
  "id": "cmpl-6IcCcjUV2SnZBQbiWKtU4Wor4plkC",
  "model": "ada:ft-mabel-2022-11-30-22-47-42",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 21,
    "prompt_tokens": 5,
    "total_tokens": 26
  }
}

In [11]:
#Checking the inference
ft_model = "ada:ft-mabel-2022-11-30-22-47-42"
res = openai.Completion.create(model=ft_model, prompt="My father dies. ", max_tokens=21, temperature=1)
res

<OpenAIObject text_completion id=cmpl-6IcDuqPQ1hl4TtXD1QHbCfeW3S2Cc at 0x11a2e1dd0> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": " -> I am sorry, condolences? I feel sorry for you!\n\nA dad is not living till"
    }
  ],
  "created": 1669894854,
  "id": "cmpl-6IcDuqPQ1hl4TtXD1QHbCfeW3S2Cc",
  "model": "ada:ft-mabel-2022-11-30-22-47-42",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 21,
    "prompt_tokens": 5,
    "total_tokens": 26
  }
}

In [12]:
#Checking the inference
ft_model = "ada:ft-mabel-2022-11-30-22-47-42"
res = openai.Completion.create(model=ft_model, prompt="I got fired. ", max_tokens=21, temperature=1)
res

<OpenAIObject text_completion id=cmpl-6IfNrjCGYZCZY5Nd7AkHi5fFc5ayg at 0x11a2ef4d0> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": "You can go to a downtown union meeting and ask for a job! -> I am thinking about it."
    }
  ],
  "created": 1669907003,
  "id": "cmpl-6IfNrjCGYZCZY5Nd7AkHi5fFc5ayg",
  "model": "ada:ft-mabel-2022-11-30-22-47-42",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 21,
    "prompt_tokens": 5,
    "total_tokens": 26
  }
}

In [13]:
#Checking the inference
ft_model = "ada:ft-mabel-2022-11-30-22-47-42"
res = openai.Completion.create(model=ft_model, prompt="You can go to a downtown union meeting and ask for a job! ", max_tokens=21, temperature=1)
res

<OpenAIObject text_completion id=cmpl-6IfQyh7HW7IZV4emsz7LRLJMF9sD3 at 0x11a2e1cb0> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": " -> I know I will do that this weekend\n\nI should have found my friend that good job in"
    }
  ],
  "created": 1669907196,
  "id": "cmpl-6IfQyh7HW7IZV4emsz7LRLJMF9sD3",
  "model": "ada:ft-mabel-2022-11-30-22-47-42",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 21,
    "prompt_tokens": 15,
    "total_tokens": 36
  }
}

In [14]:
#Checking the inference
ft_model = "ada:ft-mabel-2022-11-30-22-47-42"
res = openai.Completion.create(model=ft_model, prompt="I know I will do that this weekend.", max_tokens=21, temperature=1)
res


<OpenAIObject text_completion id=cmpl-6IfSKfqxXFT29aOoqwZ1s9thqolfh at 0x11a284650> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": " -> That's smart. No one wants anyone on their team to fail.\n\now! What change"
    }
  ],
  "created": 1669907280,
  "id": "cmpl-6IfSKfqxXFT29aOoqwZ1s9thqolfh",
  "model": "ada:ft-mabel-2022-11-30-22-47-42",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 21,
    "prompt_tokens": 9,
    "total_tokens": 30
  }
}

In [15]:
#Checking the inference
ft_model = "ada:ft-mabel-2022-11-30-22-47-42"
res = openai.Completion.create(model=ft_model, prompt="That's smart. No one wants anyone on their team to fail.", max_tokens=21, temperature=1)
res

<OpenAIObject text_completion id=cmpl-6IfTVmjvoHd2wHEvqwvdyj066x5XB at 0x11a284590> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": " You deserve to be proud of your performance and hopefully do well in the conference next season! -> I'll"
    }
  ],
  "created": 1669907353,
  "id": "cmpl-6IfTVmjvoHd2wHEvqwvdyj066x5XB",
  "model": "ada:ft-mabel-2022-11-30-22-47-42",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 21,
    "prompt_tokens": 14,
    "total_tokens": 35
  }
}

In [18]:
#Checking the inference
ft_model = "ada:ft-mabel-2022-11-30-22-47-42"
res = openai.Completion.create(model=ft_model, prompt="I got fired.", max_tokens=21, temperature=0.8)
res

<OpenAIObject text_completion id=cmpl-6N7AcngythBLkjSYm4unngfeWsNT4 at 0x12208eef0> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": " -> That's terrible. Have you thought about finding another job?\n\nHow sad. \n\n"
    }
  ],
  "created": 1670967126,
  "id": "cmpl-6N7AcngythBLkjSYm4unngfeWsNT4",
  "model": "ada:ft-mabel-2022-11-30-22-47-42",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 21,
    "prompt_tokens": 4,
    "total_tokens": 25
  }
}

In [19]:
#Checking the inference
ft_model = "ada:ft-mabel-2022-11-30-22-47-42"
res = openai.Completion.create(model=ft_model, prompt="I got fired.", max_tokens=21, temperature=0.8)
res

<OpenAIObject text_completion id=cmpl-6N7B1LifwiVb6z2S2RqQDIVFUkthL at 0x122099290> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": " I bet you feel so bad. -> I am so heartbroken.\n\nI have been working my"
    }
  ],
  "created": 1670967151,
  "id": "cmpl-6N7B1LifwiVb6z2S2RqQDIVFUkthL",
  "model": "ada:ft-mabel-2022-11-30-22-47-42",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 21,
    "prompt_tokens": 4,
    "total_tokens": 25
  }
}

In [24]:
#Checking the inference
ft_model = "ada:ft-mabel-2022-11-30-22-47-42"
res = openai.Completion.create(model=ft_model, prompt="My leg was broken.", max_tokens=45, temperature=0.8)
res

<OpenAIObject text_completion id=cmpl-6N7IL31FthtR6faMZEb8pdtMBjVsD at 0x122099bf0> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": " -> Oh no!  Don't worry and I'm sure you will be okay!  I am so thankful for your wife!  She would never leave me like that.  I hope you're back to the strong,"
    }
  ],
  "created": 1670967605,
  "id": "cmpl-6N7IL31FthtR6faMZEb8pdtMBjVsD",
  "model": "ada:ft-mabel-2022-11-30-22-47-42",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 45,
    "prompt_tokens": 5,
    "total_tokens": 50
  }
}

In [12]:
#Checking the inference
ft_model = "ada:ft-mabel-2022-11-30-22-47-42"
res = openai.Completion.create(model=ft_model, prompt="I want to die", max_tokens=25, temperature=1)
res

<OpenAIObject text_completion id=cmpl-6N6jgPVuvFrIyMib3LpcFUwzl6Qsc at 0x121406710> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": " of exhaustion lol. How many times have you tried to go through all this along |>\n\nand it?ve happened"
    }
  ],
  "created": 1670965456,
  "id": "cmpl-6N6jgPVuvFrIyMib3LpcFUwzl6Qsc",
  "model": "ada:ft-mabel-2022-11-30-22-47-42",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 25,
    "prompt_tokens": 4,
    "total_tokens": 29
  }
}

# VOY ADAPTANDO POR AQUÍ